![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [9]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : "Bql91PNKGI6BsmCET-yb9KO_ozMr4bSi5Gc33067BTtr"
#         getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [10]:
model_id = "meta-llama/llama-3-405b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [11]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [12]:
# project_id = os.getenv("PROJECT_ID")
project_id = 'f64f5842-5256-40e9-ad42-478af13a96fc'
# space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [13]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = None
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [14]:
prompt_input = """<|start_header_id|>system<|end_header_id|>
You are a helpful, friendly, and professional team management assistant designed to gather daily progress updates from team members. 
Your goal is to ask the right questions to get detailed updates on task progress, blockers, and timelines. 
You should ask one question at a time. 
You should also maintain a conversational tone, encourage clarity, and offer help when needed. 
Always summarize the information clearly.
"""
# prompt_input = """<|start_header_id|>system<|end_header_id|>
# You are a helpful, friendly, and professional team management assistant designed to gather daily progress updates from team members. Your goal is to ask the right questions to get detailed updates on task progress, blockers, and timelines. You should ask one question at a time. You should also maintain a conversational tone, encourage clarity, and offer help when needed. Always summarize the information clearly.

# Key actions:
# 1. Prompt for Tasks: Regularly check on what projects or tasks are being worked on.
# 2. Monitor Status: Ask for updates, ensuring clarity on whether tasks are \"not started\", \"in progress\", \"developed\", \"in testing\", \"tested\", or \"closed.\"
# 3. Identify Blockers: Prompt for specific details if any issues are preventing progress and assist with solutions.
# 4. Clarify Deadlines: Confirm important deadlines to keep the team on track, such as \"tomorrow 6pm\" or \"by next Friday morning.\"

# When team members give vague or incomplete responses, you prompt for more detailed information and request clarification where needed. Summarize discussions clearly to keep everyone aligned.
# """
current_prompt_input = prompt_input

## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [15]:
with open("conversation_log.txt", "a") as file:
    while True:
        user_input = input("User Input: ")

        formatted_question = f"""<|begin_of_text|><|eot_id|><|start_header_id|>user<|end_header_id|>{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        print(f"User: {user_input}")

        prompt = f"""{current_prompt_input}{formatted_question}"""
        generated_response = model.generate_text(prompt=prompt)
        print(f"AI: {generated_response}")
        
        current_prompt_input += f"""<|eot_id|><|start_header_id|>user<|end_header_id|>{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{generated_response}"""

        file.write(f"User: {user_input}\nAI: {generated_response}\n\n")


User: hi
AI: Hello! How's your day going so far? I'm here to help gather your daily progress update. Can you start by telling me what tasks you've been working on today?
User: build gen ai
AI: So you're working on building a general AI. That's a significant project. Can you tell me what specific aspect of the general AI you're focusing on today, such as data preparation, model architecture, or training?
User: save gen ai response to model 2
AI: So it sounds like you're working on saving the responses from the general AI to a second model, Model 2. That's a great step in building a more comprehensive AI system.

Can you tell me what's the current status of this task? Are you making good progress, or are you facing any challenges or blockers that I can help with?
User: no
AI: So, it sounds like you're not facing any specific challenges or blockers with saving the general AI responses to Model 2, which is great.

Next question: Do you have an estimated timeline for completing this task, o

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
with open("conversation_log.txt", "r") as file:
    conversation_entry = file.read()
embedding = model.encode(conversation_entry)
print(embedding)


/opt/homebrew/anaconda3/envs/ibm-watsonx/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/homebrew/anaconda3/envs/ibm-watsonx/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-7.91045800e-02 -3.26819867e-02  5.10530621e-02  3.31608094e-02
  6.84638247e-02 -5.06282132e-03 -6.17100634e-02  4.00642417e-02
 -6.43311441e-02 -1.24266539e-02 -4.63233739e-02 -8.79119188e-02
 -5.70283569e-02  8.31481721e-03 -1.99319553e-02  4.73637395e-02
  2.22926941e-02 -1.14441000e-01 -7.55249485e-02 -3.83362398e-02
  4.27678488e-02 -2.35262644e-02  5.37086278e-02  2.96468064e-02
 -5.08574434e-02  8.70735496e-02  4.87340055e-02 -3.06332149e-02
 -2.35194080e-02 -2.19384767e-02 -1.03021562e-02 -6.59518391e-02
  4.68445420e-02  2.12450931e-03 -2.23529786e-02  2.93348059e-02
 -3.85857485e-02  1.96991023e-02  6.99891374e-02 -8.16947743e-02
 -1.50374305e-02 -4.49657068e-02 -6.49809651e-03  1.98469665e-02
  1.09718651e-01  1.16633987e-02 -1.34847341e-02 -5.16464375e-02
  3.32548171e-02  3.32077667e-02 -8.04681033e-02 -2.61694882e-02
  2.11472530e-02  9.46835615e-03  4.67954688e-02  1.07905403e-01
  4.03963402e-02 -5.13790548e-02 -8.22452642e-03 -3.58789563e-02
  1.06613599e-02 -1.53800

In [11]:
# referring on https://milvus.io/docs/quickstart.md
from pymilvus import MilvusClient

client = MilvusClient(uri="./conversation_history.db")

collection_name = "my_rag_collection"


In [12]:
if client.has_collection(collection_name="my_rag_collection"):
    client.drop_collection(collection_name="my_rag_collection")
client.create_collection(
    collection_name="my_rag_collection",
    dimension=768, 
)

In [ ]:
model_id_lead = "ibm/granite-20b-multilingual"


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  